# Laborarbeit KI - CSP
**Belegung von Laboren für Projektpräsentationen**

Zuerste müssen alle benötigten Bibliotheken installiert werden

In [98]:
!pip install -r requirements.txt

Jetzt werden alle benötigten Bibliotheken importiert und ein Datensatz als DataFrame geladen. Um einen ersten Überblick zu bekommen.

In [99]:
from constraint import *
import pandas as pd
import itertools
from functools import partial

problem1 = pd.read_csv('./data/DS_CSP_1/pr_conf_004.txt', sep=';')
problem1

,Studiengang,Projektgruppen,Kommissionen
0,Informatik,4,3
1,Elektrotechnik,2,2
2,WIW,2,1
3,Maschinenbau,2,2
4,Data Science,1,3


Wir haben in diesem Datensatz 5 verschiedene Studiengänge und dabei hat Informatik die meisten Projektgruppen.

## Umgebung definieren
Jetzt werden die Umgebungsvariablen definiert (Räume, Solts, ...).

In [100]:
DAYS = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
SLOTS = ['S1', 'S2', 'S3', 'S4']
SLOT_TIMES = {'S1': '08:00 - 09:00', 'S2': '10:00 - 11:00', 'S3': '13:00 - 14:00', 'S4': '15:00 - 16:00'}
ROOMS = ['L1', 'L2', 'L3']
AMOUNT_PRESENTATIONS = 3
MAX_SWICHTES = 1

ACROS = {
    'Informatik': 'INF',
    'Data Science': 'DS',
    'Elektrotechnik': 'ET',
    'WIW': 'WIW',
    'Maschinenbau': 'MB',
    'Data Science': 'DS'
}

MAX_SLOTS = len(DAYS) * len(SLOTS) * len(ROOMS)
MAX_SLOTS

60

Es gibt insgesamt 60 Slots, verteilt über drei Räume von Mo. - Fr.

## Hilfsfunktionen
Hier werden zuerst Hilfsfunktionen definert, um die spätere Problem-Funktion nicht zu überfüllen.
- `getDomain()`: Erstellt die Domain für den jeweiligen Studiengang
- `getVariables()`: Erstellt für den jeweiligen Studiengang eine Liste von allen Gruppen, mit ihren 3 Präsentationen

In [101]:
def getDomain(study_course: str, amount_committees: int):
    """
    Create the domian for the presentation variables, based on days, slots, rooms and committees.
    
    Returns: a list of tuples (day, slot, room, committee)
    """
    domain = []
    for day in DAYS:
        for slot in SLOTS:
            for room in ROOMS:
                for committee in range(amount_committees):
                    domain.append((day, slot, room, f"{study_course}_K{committee}"))
    return domain

def getVariables(study_course: str, amount_groups: int):
    """
    Defines all variables for a given study course and amount of groups.
    
    Returns: a list of variable names
    """
    variables = []
    for group in range(1, amount_groups + 1):
        for p in range(1, AMOUNT_PRESENTATIONS + 1):
            var_name = f"{study_course}_{group}_P{p}"  # Unique name - e.g. "INF_1_P1"
            variables.append(var_name)
    return variables

## Constraints
Jetzt müssen die jeweiligen constraints erstellt werden. Dafür wird hier jedes Constraint als Funktion definiert und später zum Projekt hinzugefügt.

**Constraints**:
- `unique_appointment`: Jede Präsentation braucht einen anderen Termin (Tag / Zeit)
- `group_clash`: Eine Gruppe kann nicht mehrere Präsentationen halten in einem Zeitslot
- `comitee_clash`: Eine Komission kann nicht mehreren Präsenationen gleichzeitig zuhören
- `max_switches`: Jede Komission darf **max** einmal am Tag den Raum wechseln
- `rest_time`: Wenn eine Gruppe den letzten Slot hat, darf sie nicht den ersten Slot am direkt nächsten Tag haben

Der weiche Contraint kann nicht als "normaler" Constraint angegeben werden, sondern muss Programmtechnisch umgesetzt werden.

In [102]:
def unique_appointment(d1: tuple, d2: tuple):
    """
    Constraint: Ensure that two presentations do not occur at the same time (day and slot) in the same room.
    """
    day1, slot1, room1, _ = d1
    day2, slot2, room2, _ = d2
    return not (day1 == day2 and slot1 == slot2 and room1 == room2)


def group_clash(d1: tuple, d2: tuple):
    """
    Constraint: Ensure that a group does not have overlapping presentations.
    Careful: The inputs need to be from the same group!
    """
    day1, slot1, _, _ = d1
    day2, slot2, _, _ = d2
    return not (day1 == day2 and slot1 == slot2)
    

def committee_clash(d1: tuple, d2: tuple):
    """
    Constraint: Ensure that a committee does not have overlapping presentations.
    """
    day1, slot1, _, committee1 = d1
    day2, slot2, _, committee2 = d2
    if committee1 == committee2:
        return not (day1 == day2 and slot1 == slot2)
    return True


def max_switches(*args, target_committee, target_day):
    """
    Constraint: Ensure that a committee does not have overlapping presentations.
    """
    # filter for target commitee and day
    relevant_sessions = []
    for val in args:
        day, slot, room, committee = val
        if day == target_day and committee == target_committee:
            relevant_sessions.append((slot, room))
    
    # only one or no sessions -> can't switch
    if len(relevant_sessions) <= 1:
        return True
    
    # sort by slot
    relevant_sessions.sort(key=lambda x: x[0])
    
    # count switches
    switches = 0
    current_room = relevant_sessions[0][1]
    
    for i in range(1, len(relevant_sessions)):
        next_room = relevant_sessions[i][1]
        if next_room != current_room:
            switches += 1
            current_room = next_room
            
    return switches <= 1


def rest_time(d1: tuple, d2: tuple):
    """
    Constraint: Ensure that when the group has the last slot, that it can have the first at the direct next day.
    """
    day1, slot1, _, _ = d1
    day2, slot2, _, _ = d2
    
    # Get indices for days and slots (0-4 for days, 0-3 for slots)
    day_index1 = DAYS.index(day1)
    day_index2 = DAYS.index(day2)
    slot_index1 = SLOTS.index(slot1)
    slot_index2 = SLOTS.index(slot2)
    
    # Day1 is day n  (slot 4) and Day2 is day n+1 (slot 1)
    if day_index1 + 1 == day_index2 and slot_index1 == 3 and slot_index2 == 0:
        return False 
    
    # Day1 is day n+1 (slot 1) and Day2 is day n (slot 4)
    if  day_index1 == day_index2 + 1 and slot_index2 == 0 and slot_index1 == 3:
        return False 
    
    return True

## Problem definieren
Hier wird eine Funktion definiert, welche das Problem repräsentiert. Dabei werden die Variablen geladen und die Constraints hinzugefügt, welche zuvor definiert wurden.

Zuerst werden für jede Präsentation von jeder Gruppe eine Varibale hinzugefügt, mit der Domäne `(Tag, Slot, Raum, Komissionen)`. 

In [ ]:
def presentation_problem(input_data: pd.DataFrame, solver: str = 'backtracking') -> dict:
    problem = Problem()
    
    # --- Define variables ---
    variables = []
    vars_by_course: dict = {}
    for study in input_data.itertuples():
        # get variables and domain for each study course
        vars = getVariables(ACROS[study[1]], study[2])
        domain = getDomain(ACROS[study[1]], study[3])
        variables.extend(vars)
        vars_by_course[ACROS[study[1]]] = vars
        
        problem.addVariables(vars, domain)
    
    # --- little check ---
    if len(variables) > MAX_SLOTS:
        print("Too many variables for available slots!")
        return None
    elif len(variables) == MAX_SLOTS:
        print("Warning: Variables equal to available slots, no freedom in scheduling.")
    
    
    # --- Define Constraints ---
    
    # Unique appointments - on all variables
    for i in range(len(variables)):
        for j in range(i + 1, len(variables)):
            problem.addConstraint(unique_appointment, (variables[i], variables[j]))
    
    # group constraints - group clash and rest time
    for idx in range(0, len(variables), 3):
        # split into list of group presentations - can split by AMOUNT_PRESENTATIONS
        group_vars = variables[idx:idx + AMOUNT_PRESENTATIONS]

        # iter over all group combinations
        for g1, g2 in itertools.combinations(group_vars, 2):
            # Add group clash constraint       
            problem.addConstraint(group_clash, (g1, g2))

            # Add rest time constraint
            problem.addConstraint(rest_time, (g1, g2))

    # comitee constraints - comitee clash and max switches
    for course, vars in vars_by_course.items():
        full_course_name = next((key for key,value in ACROS.items() if value == course), None)
        
        # comitee clash - for each variable pair in a study course
        for var1, var2 in itertools.combinations(vars, 2):
            problem.addConstraint(committee_clash, (var1, var2))
            
        # max switches - for each committee and day
        # go over all committees
        for i in range(input_data.loc[input_data['Studiengang'] == full_course_name, 'Kommissionen'].values[0]):
            commitee = f"{course}_K{i}"   # Need the same format as in domain
            
            # for each day add the constraint
            for day in DAYS:
                # partial function to add traget committee and day into the max_switches function
                problem.addConstraint(partial(max_switches, target_committee=commitee, target_day=day), vars)
                
        
    # --- Solve the problem ---  
    return problem.getSolution()
    

## Ausgabe formatieren
Um nicht ein Dictionary anschauen zu müssen, wird eine anschauliche Tabelle erstellt mithilfe von der Pandas `pivot_table`. Dabei repräsentieren die Spalten die jeweiligen Tage und die Zeilen die Räume mit ihren Zeitslots. 

In [114]:
def schedule_table(solution: dict) -> pd.DataFrame:
    """
    Creates a table with slots as rows and days as columns.
    Cell = "Group Committee Room"
    """
    
    # create formatted data
    formatted_data = []
    for var, (day, slot, room, committee) in solution.items():
        course, group, presentation = var.split('_')
        label = f"{course}-{group} ({presentation}) - [{committee}]"    # e.g. "INF-1 (P1) - [INF_K0]"
        
        formatted_data.append({
            'Tag': day,
            'Slot': SLOT_TIMES.get(slot, slot), # convert slot to time range
            'Raum': room,
            'Belegung': label
        })
        
    df = pd.DataFrame(formatted_data)
    
    # create table for display
    table = df.pivot_table(
        index=['Raum', 'Slot'], 
        columns='Tag', 
        values='Belegung', 
        aggfunc='first'
    )
    # Reindex to ensure all days and slots/rooms are present and fill missing
    table = table.reindex(columns=DAYS, index=pd.MultiIndex.from_product(
        [ROOMS, [SLOT_TIMES[s] for s in SLOTS]], names=['Raum', 'Uhrzeit'])).fillna('---')
    
    return table